In [1]:
import xarray as xr
import dask
import numpy as np
import os
import time
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=1,name='make_profiles',walltime='00:20:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1','--ntasks-per-node=24'],memory='120GB',interface='ib0') 
print(cluster.job_script()) 

cluster.scale(240) 



#!/usr/bin/env bash

#SBATCH -J make_profiles
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=112G
#SBATCH -t 00:20:00
#SBATCH --constraint=HSW24
#SBATCH --exclusive
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=24
JOB_ID=${SLURM_JOB_ID%;*}



/scratch/cnt0024/hmg2840/albert7a/anaconda3/bin/python -m distributed.cli.dask_worker tcp://172.30.100.2:45713 --nthreads 0 --nprocs 28 --memory-limit 4.29GB --name make_profiles--${JOB_ID}-- --death-timeout 60 --local-directory $SCRATCHDIR/daskjobqueue --interface ib0



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
cluster

In [3]:
from dask.distributed import Client
client = Client(cluster)


In [4]:
client

Client Scheduler: tcp://172.30.100.2:45713 Dashboard: http://172.30.100.2:46038/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)


In [6]:
def time_filt_aco():

    fileUtide="/store/albert7a/eNATL60/eNATL60-BLBT02-S/1h/surf/eNATL60ACO-BLBT02_y2009m10d29.1h_sozocrtx.nc"
    fUtide = xr.open_mfdataset(fileUtide,chunks={'x':500,'y':500})
    Utide=fUtide['sozocrtx'][:]
    navlat= fUtide['nav_lat'][0]
    navlon= fUtide['nav_lon'][0]
    T=2*np.pi/(1E-4)

    wutide=Utide.window
    wutide.set(n=48,dim='time_counter', cutoff=2*T)
    Utidefilt=wutide.convolve()
    output_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/scale/eNATL60ACO-BLBT02_y2009m10d29.1h_sozocrtx_time_filt_2Coriolis.nc'
    if not os.path.exists(output_name):
            print('to dataset')
            dataset=Utidefilt.to_dataset(name='uprime')
            dataset['uprime'].attrs=Utide.attrs
            dataset['uprime'].attrs['standard_name']='no wave zonal velocity' 
            dataset['uprime'].attrs['long_name']='uprime'
            dataset['uprime'].attrs['units']='m/s'        
            dataset.attrs['global_attribute']= 'no wave zonal velocity computed on occigen with dask-jobqueue '+str(today)
            print('to netcdf')
            dataset.to_netcdf(path=output_name,mode='w')
            print('all done')
    print('already done')


In [7]:
%time time_filt_aco()

to dataset
to netcdf
all done
already done
CPU times: user 1.17 s, sys: 226 ms, total: 1.39 s
Wall time: 7.96 s


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")
